# Data Pre-Processing

#### Change Log
* Modules
* Separated from Gensim_Test_ver5
* Exporting to private repo in Github

### 2016-07-11
* added Indian Names as stoplist. (Output needs verification)
* Introducing Spell check before tokenizer 
* Skip - and . to preserve hyphenated words 

In [ ]:
import logging
logging.basicConfig(filename='./logfile.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [1]:
# Import Stuffs
import gensim
from gensim import *
import re, os, string, timeit 
from nltk.stem.porter import PorterStemmer
#table = string.maketrans("","")
p_stemmer = PorterStemmer()

In [2]:
# Stop_list
## To do: Import English stoplist from nltk
#Todo: Add this as a separate file
# Todo: Add nameslist as stop parameter. Avaialble in indian_names.txt

stoplist=set('''court supreme in as xx xxx
                on or by that is was are were no not vs be being been has have had case 
                he she I one every least less many now ever never say says said also get
                go goes just made make put see seen whether like well back even still way
                take since another however two three four five six seven eight nine ten 
                first second new old high 
                long and but if or because as until while  of at by for with about against
                between into through during before after above below to from up down in out
                on off over under  again further then once  here there when where why how
                all any both each few more most other some such  no nor not only own same
                so than too very what which who whom this that these those they them their
                theirs themselves it its itself she her herself he his him himself you your
                yours yourself yourselves our ours ourselves us thou we me my myself i am 
                is are was were be been being have has had having do does did doing will 
                would shall should can could may might must ought a an the'''.split())

In [3]:
# Stop_list: Indian Names
names = open('../indian_names.txt','rb').read().split()
nameset=set(names)
stoplist |= nameset

In [4]:
# Stop_list: Indian City Names

In [5]:
# Spell Check module
from enchant import DictWithPWL ##Needs pyenchant installed
from enchant.checker import SpellChecker
my_dict = DictWithPWL("en_US", "../indian_names.txt")
chkr = SpellChecker(my_dict)

## Gensim Module


### Tokenization, Corpus and Dictionary Generation

In [ ]:
def iter_documents(top_directory):
    """Iterate over all documents, yielding a document (=list of utf8 tokens) at a time."""
    for root, dirs, files in os.walk(top_directory):
        for file in filter(lambda file: file.endswith('.txt'), files):
            document = open(os.path.join(root, file)).read() # read the entire document, as one big string
            #document_clean = document.translate(table, '!@#$%^&*()_-+={}[]:;"\'|<>,.?/~`1234567890')
            document_clean = re.sub('[^a-zA-Z0-9]', ' ', document) #Replaces any non-alphanumeric char to space
            #wrong_spells(document_clean)
            yield [i for i in gensim.utils.tokenize(document_clean, lower=True)] 
            
class MyCorpus(object):
    def __init__(self, top_dir):
        self.top_dir = top_dir
        self.dictionary = gensim.corpora.Dictionary(iter_documents(top_dir))
        self.dictionary.filter_extremes(no_below=2, keep_n=30000) # check API docs for pruning params
        stop_ids = [self.dictionary.token2id[stopword] for stopword in stoplist if stopword in self.dictionary.token2id]
        wrong_spell_ids = [chkr.check(word) for word in self.dictionary.token2id]
        self.dictionary.filter_tokens(stop_ids)
        self.dictionary.filter_tokens(wrong_spell_ids)
        self.dictionary.compactify()
        self.dictionary.save('texts2800dict.dict')
    def __iter__(self):
        for tokens in iter_documents(self.top_dir):
            yield self.dictionary.doc2bow(tokens)

corpus = MyCorpus('texts_redo') # create a dictionary
#for vector in corpus: # convert each document to a bag-of-word vector
 #   print vector
corpora.MmCorpus.serialize('ser2800corpus.mm', corpus) #Multiple formats available

#### Output 
* Serialized Corpus
* dictionary

In [ ]:
sercorpus = corpora.MmCorpus('ser2800corpus.mm')
dictionary= corpora.Dictionary.load('texts2800dict.dict')
print(dictionary)